In [1]:
using Pkg
Pkg.activate("/media/mat/HDD/AdaptiveTransportMap/")

 Activating environment at `/media/mat/HDD/AdaptiveTransportMap/Project.toml`


In [3]:
using Revise
using AdaptiveTransportMap
using LinearAlgebra
import LinearAlgebra: ldiv!, dot
using Test
using ForwardDiff
using SpecialFunctions
using BenchmarkTools
using TransportMap
using QuadGK
using Polynomials
using Distributions
using Random
using LoopVectorization
using Optim
using NLsolve
using MLDataUtils
using MLDataPattern
using Test
using SparseArrays
using AdaptiveTransportMap: vander, transform!, evaluate, ncoeff, optimize, negative_log_likelihood!, derivative

# using Profile
# using ProfileView
using StaticArrays
using QRupdate

┌ Info: Precompiling AdaptiveTransportMap [bdf749b0-1400-4207-80d3-e689c0e3f03d]
└ @ Base loading.jl:1278


In [4]:
m = 100
n = 5

A = randn(m, n)
F = qr(A)

Q1 = F.Q*Matrix(1.0*I, m, n)

@show norm(Q1 - F.Q*Matrix(1.0*I, m, m)[:,1:n])

Q2 = F.Q * Matrix(1.0*I, m, m-n)

# @show norm(Q2 - F.Q*Matrix(1.0*I, m, m)[:,n+1:m])
v = randn(m);

norm(Q1 - F.Q * (Matrix(1.0I, m, m))[:, 1:n]) = 0.0


In [5]:
qr(hcat(A, reshape(v,(m,1)))).R

6×6 Array{Float64,2}:
 10.1498  -0.444181   -1.93539   -2.73847     0.815124  -0.0982151
  0.0      9.9724     -0.637325   0.173324   -0.420596   0.115398
  0.0      0.0       -10.0452     2.10551    -0.218037  -1.54195
  0.0      0.0         0.0       -9.591       1.01912    2.14354
  0.0      0.0         0.0        0.0       -10.4777     0.727149
  0.0      0.0         0.0        0.0         0.0       11.4623

In [11]:
qraddcol(A, F.R, v)

Variables
  #self#::Core.Compiler.Const(QRupdate.qraddcol, false)
  A::Array{Float64,2}
  Rin::Array{Float64,2}
  a::Array{Float64,1}

Body::Array{Float64,2}
1 ─ %1 = (#self#)(A, Rin, a, 0.0)::Array{Float64,2}
└──      return %1


In [14]:
function timing()
    @btime begin 
        m = 1000
        n = 50

        A = randn(m, n)
        qr(A)
        for i=1:20
            v = randn(m)
            A = hcat(A, reshape(v,(m,1)))
        end
    end
    @btime begin 
        m = 1000
        n = 50

        A = randn(m, n)
        qr(A)
        for i=1:20
            v = randn(m)
            A = hcat(A, reshape(v,(m,1)))
            qr(A)
        end
    end
    @btime begin
                m = 1000
        n = 50

        A = randn(m, n)
        F = qr(A)
        R = F.R
        for i=1:20
            v = randn(m)
            R = qraddcol(A, R, v)
            
            A = hcat(A, reshape(v,(m,1)))
        end
    end
end

timing (generic function with 1 method)

In [15]:
timing()

  1.364 ms (86 allocations: 10.18 MiB)
  17.929 ms (194 allocations: 20.08 MiB)
  2.417 ms (368 allocations: 11.47 MiB)


In [16]:
F.R

5×5 Array{Float64,2}:
 10.1498  -0.444181   -1.93539   -2.73847     0.815124
  0.0      9.9724     -0.637325   0.173324   -0.420596
  0.0      0.0       -10.0452     2.10551    -0.218037
  0.0      0.0         0.0       -9.591       1.01912
  0.0      0.0         0.0        0.0       -10.4777

In [3]:
Nx = 2
Ne = 8
m = 20

X =  Matrix([0.267333   1.43021;
      0.364979   0.607224;
     -1.23693    0.249277;
     -2.0526     0.915629;
     -0.182465   0.415874;
      0.412907   1.01672;
      1.41332   -0.918205;
      0.766647  -1.00445]');

# X = randn(Nx, Ne)

idx = [0 0; 0 1; 1 0]

Nψ = 4#size(idx,1)

coeff = [-0.28829821609289064;
         -1.1768877197305714;
          0.40090743090076547];
#           1.101330816362896];
C = MapComponent(m, Nx, idx, coeff);

addedidx = reshape([1 1], (1,2))
addedcoeff = [0.1910678298362849]

1-element Array{Float64,1}:
 0.1910678298362849

In [4]:
fnew = ParametricFunction(ExpandedFunction(C.I.f.f.B, vcat(getidx(C), addedidx), vcat(getcoeff(C), addedcoeff)))

ParametricFunction(ExpandedFunction(20, 4, 2, MultiBasis(Basis of 20 functions: Constant -> 18th degree Probabilistic Hermite function
, 2), [0 0; 0 1; 1 0; 1 1], [-0.28829821609289064, -1.1768877197305714, 0.40090743090076547, 0.1910678298362849]))

In [5]:
ψ = evaluate_basis(fnew.f, X, getidx(C))
addedψ = evaluate_basis(fnew.f, X, addedidx)
ψnew = evaluate_basis(fnew.f, X)

8×4 Array{Float64,2}:
 1.0  0.378763  0.620434  0.234997
 1.0  0.575999  0.610931  0.351895
 1.0  0.621883  0.430862  0.267945
 1.0  0.512188  0.220301  0.112835
 1.0  0.604891  0.626383  0.378894
 1.0  0.487776  0.605263  0.295233
 1.0  0.511584  0.383338  0.19611
 1.0  0.490809  0.545307  0.267642

In [6]:
F = qr(ψ)

LinearAlgebra.QRCompactWY{Float64,Array{Float64,2}}
Q factor:
8×8 LinearAlgebra.QRCompactWYQ{Float64,Array{Float64,2}}:
 -0.353553   0.698591    0.219991   -0.202025   …  -0.23307     -0.335742
 -0.353553  -0.256782    0.30033     0.383758       0.0879981   -0.130855
 -0.353553  -0.479033   -0.139451   -0.672087      -0.391967    -0.0835975
 -0.353553   0.0523058  -0.740525    0.472914      -0.297757    -0.0766019
 -0.353553  -0.396729    0.35552     0.248427       0.0601942   -0.10873
 -0.353553   0.170554    0.238827    0.0632315  …  -0.00488694  -0.0913354
 -0.353553   0.055234   -0.320581   -0.26452        0.830306    -0.0842048
 -0.353553   0.15586     0.0858895  -0.0296987     -0.0508178    0.911067
R factor:
3×3 Array{Float64,2}:
 -2.82843  -1.47923   -1.42935
  0.0      -0.206449   0.0425696
  0.0       0.0        0.387938

In [7]:
?QR

search: quantile_ricardo QR qr qr! QRPivoted qrnegative_log_likelihood! sqrt



```
QR <: Factorization
```

A QR matrix factorization stored in a packed format, typically obtained from [`qr`](@ref). If $A$ is an `m`×`n` matrix, then

$$
A = Q R
$$

where $Q$ is an orthogonal/unitary matrix and $R$ is upper triangular. The matrix $Q$ is stored as a sequence of Householder reflectors $v_i$ and coefficients $\tau_i$ where:

$$
Q = \prod_{i=1}^{\min(m,n)} (I - \tau_i v_i v_i^T).
$$

Iterating the decomposition produces the components `Q` and `R`.

The object has two fields:

  * `factors` is an `m`×`n` matrix.

      * The upper triangular part contains the elements of $R$, that is `R = triu(F.factors)` for a `QR` object `F`.
      * The subdiagonal part contains the reflectors $v_i$ stored in a packed format where $v_i$ is the $i$th column of the matrix `V = I + tril(F.factors, -1)`.
  * `τ` is a vector  of length `min(m,n)` containing the coefficients $au_i$.


In [ ]:
F.Q * Matrix(I, size(F.Q, 1), nx),

In [73]:
function timing()
    @btime begin 
        A = randn(1000, 50)
        v = randn(1000,1)
        qr(A)
        qr(hcat(A,v))
    end
    
    @btime begin
        A = randn(1000, 50)
        v = randn(1000,1)
        F = UpdatableQR(A)
        add_column_householder!(F,v[:,1])    
    end
end

timing (generic function with 1 method)

In [74]:
timing()

  1.421 ms (13 allocations: 1.61 MiB)
  5.024 ms (29 allocations: 17.44 MiB)


LinearAlgebra.QRCompactWY{Float64,Array{Float64,2}}
Q factor:
950×950 LinearAlgebra.QRCompactWYQ{Float64,Array{Float64,2}}:
 -0.00219086   -0.0109498     0.00160354   …   0.0260315    -0.0442697
 -0.0109498     0.99988       1.75201e-5       0.000284418  -0.000483687
  0.00160354    1.75201e-5    0.999997        -4.16513e-5    7.0833e-5
  0.013761      0.000150352  -2.20181e-5      -0.000357438   0.000607866
 -0.063161     -0.000690091   0.00010106       0.00164058   -0.00279001
  0.0471475     0.000515129  -7.54375e-5   …  -0.00122464    0.00208264
  0.069209      0.000756171  -0.000110737     -0.00179768    0.00305717
 -0.00137804   -1.50563e-5    2.20491e-6       3.5794e-5    -6.0872e-5
  0.0275082     0.000300552  -4.4014e-5       -0.000714515   0.00121512
  0.043181      0.000471792  -6.90911e-5      -0.00112161    0.00190743
 -0.000328639  -3.59068e-6    5.25833e-7   …   8.53627e-6   -1.45169e-5
  0.0580574     0.00063433   -9.28937e-5      -0.00150802    0.00256457
  0.0527232  

In [16]:
addedψ = evaluate_basis(fnew.f, X, addedidx)[:,1]

8-element Array{Float64,1}:
 0.2349974942990817
 0.351895438453913
 0.267945347873556
 0.11283549249688966
 0.37889365830058164
 0.2952326851102279
 0.19610952117418562
 0.26764189906926694

In [18]:
function timing()
    @btime begin
        X = randn(2,1000)
        ψ = evaluate_basis(fnew.f, X, getidx(C))
        addedψ = evaluate_basis(fnew.f, X, addedidx)[:,1]
        ψnew = evaluate_basis(fnew.f, X)
    end
    @btime begin
        X = randn(2,1000)
        ψ = evaluate_basis(fnew.f, X, getidx(C))
        addedψ = evaluate_basis(fnew.f, X, addedidx)[:,1]
        ψnew = evaluate_basis(fnew.f, X)
        qr(ψ)
        qr(ψnew)
    end
    
    @btime begin
        X = randn(2,1000)
        ψ = evaluate_basis(fnew.f, X, getidx(C))
        addedψ = evaluate_basis(fnew.f, X, addedidx)[:,1]
        ψnew = evaluate_basis(fnew.f, X)
        F =  UpdatableQR(ψ)
        add_column_householder!(F, addedψ)
    end
end
# timing()

timing (generic function with 1 method)

In [19]:
timing()

  41.095 μs (27 allocations: 252.13 KiB)
  75.377 μs (35 allocations: 307.69 KiB)
  3.019 ms (51 allocations: 16.53 MiB)


LinearAlgebra.QRCompactWY{Float64,Array{Float64,2}}
Q factor:
997×997 LinearAlgebra.QRCompactWYQ{Float64,Array{Float64,2}}:
 -0.0287529     0.00401395    0.00602951   …  -0.00838241    0.00172087
  0.00401395    0.999984     -2.35257e-5       3.27062e-5   -6.71443e-6
  0.00602951   -2.35257e-5    0.999965         4.91292e-5   -1.0086e-5
 -0.0185508     7.23808e-5    0.000108726     -0.000151154   3.10312e-5
  0.00277465   -1.0826e-5    -1.62622e-5       2.26082e-5   -4.64135e-6
 -0.0267242     0.000104271   0.00015663   …  -0.000217752   4.47035e-5
 -0.00134167    5.23487e-6    7.86349e-6      -1.09321e-5    2.2443e-6
 -0.0140031     5.46366e-5    8.20717e-5      -0.000114099   2.34239e-5
  0.000514521  -2.00754e-6   -3.0156e-6        4.19238e-6   -8.60677e-7
 -0.00114365    4.46227e-6    6.70294e-6      -9.31863e-6    1.91307e-6
  0.0051671    -2.01608e-5   -3.02843e-5   …   4.21022e-5   -8.64338e-6
  0.0236133    -9.21334e-5   -0.000138397      0.000192404  -3.94996e-5
 -0.0381551   

In [70]:
n = 600
m = 200

A = rand(n,m)
v = randn(m)

@time F = qr(A);

Q1 = Q[:,1:m]
Q2 = Q[:,m+1:n]

@time (F.Q' * Matrix(1.0*I,n-(m+1),m))*v
# @time (Matrix(1.0*I,n,m) * F.Q' * Matrix(1.0*I,n,m))*v

@show norm(Q1'*v)# -  (F.Q' * Matrix(1.0*I,n,m))*v)

  0.003614 seconds (7 allocations: 1.026 MiB)


DimensionMismatch: DimensionMismatch("first dimensions of C, 399, and V, 600 must match")

In [68]:
LAPACK

600-element Array{Float64,1}:
  0.3393175502627091
 -0.07923739197970618
 -0.04793564818244544
  0.5023994257609604
 -0.5879280814284876
 -0.2542295709677928
 -0.2827838225106669
  0.34285016529561674
  0.04446674402999848
 -0.23872684384508705
 -0.16653738974274526
  0.3206629597611629
  0.5325823729491732
  ⋮
  0.7790781084491133
 -0.847503110207023
 -0.31775625941225233
  0.1357100432473547
  0.5333629346864788
 -0.15669164303620292
  0.014684861419618875
  0.4633451181623947
 -0.6122907532137764
  0.44334346402739216
  0.34538720884302915
 -0.16477268661352273

In [57]:
F.Q * Matrix(I, size(F.Q, 1), m)

600×200 Array{Float64,2}:
 -0.0523633    -0.0418734    0.0283308   …  -0.0388924   -0.0318423
 -0.00622382   -0.0913915   -0.00518148      0.0425376    0.0644839
 -0.0297033     0.00468841  -0.0733061      -0.0502895   -0.0646676
 -0.000861618  -0.0608094   -0.0164761       0.0564105   -0.0414871
 -0.0340149     0.0354227   -0.0841016      -0.0694759   -0.0033802
 -0.0524819     0.0358096    0.0219657   …   7.60299e-5  -0.0334316
 -0.0586049    -0.0232633   -0.0138117       0.0571231   -0.00793514
 -0.0443538     0.00927013  -0.0439779       0.0167563    0.0132112
 -0.0280478     0.0265482    0.0159112      -0.0106056    0.0210681
 -0.0511942    -0.0341468    0.0737488       0.0109136    0.0123917
 -0.040626     -0.0514616   -0.0374266   …   0.0461387   -0.0650454
 -0.0412411    -0.0395853   -0.0129351       0.011246     0.0475822
 -0.0491494     0.0351298   -0.0556724       0.0614526   -0.0595294
  ⋮                                      ⋱               
 -0.0168578    -0.0681455    0.

In [58]:
Q1

600×200 Array{Float64,2}:
 -0.0218308   -0.00600274  -0.0877685   …  -0.0275777    -0.0181406
 -0.0187938   -0.0604356   -0.0513703       0.0813879     0.0400525
 -0.00292182  -0.0236161   -0.0672253      -0.0646949    -0.026167
 -0.0320766    0.00996783  -0.0190459       0.0123922    -0.0436413
 -0.0523644   -0.0437903    0.0422649      -0.0201705    -0.00281764
 -0.0560545   -0.0201515    0.0163621   …  -0.000560198  -0.0899267
 -0.0280973   -0.0357981    0.0301558      -0.0659816     0.0245865
 -0.0308715   -0.067648     0.0025547      -0.00426471    0.0275778
 -0.0133893   -0.0906553   -0.0583811      -0.015446      0.0134348
 -0.0237936   -0.0340595    0.0393902      -0.0588948     0.0199524
 -0.0648887    0.00188695   0.00248847  …  -0.0353354    -0.00136956
 -0.0425794   -0.0547981    0.0784          0.0760237    -0.0611772
 -0.00104564  -0.0467049   -0.0858344      -0.0297034     0.0199021
  ⋮                                     ⋱                
 -0.060691    -0.00290695   0.0

In [50]:
Q = F.Q*Matrix(1.0*I,n,n)

600×600 Array{Float64,2}:
 -0.0218308   -0.00600274  -0.0877685   …  -0.0488984   -0.0507896
 -0.0187938   -0.0604356   -0.0513703      -0.024539    -0.00930001
 -0.00292182  -0.0236161   -0.0672253      -0.0300917    0.0236193
 -0.0320766    0.00996783  -0.0190459       0.00494316  -0.0405474
 -0.0523644   -0.0437903    0.0422649      -0.0423611    0.0127439
 -0.0560545   -0.0201515    0.0163621   …   0.0125501   -0.0476772
 -0.0280973   -0.0357981    0.0301558      -0.072364    -0.00201243
 -0.0308715   -0.067648     0.0025547       0.0211651   -0.0757508
 -0.0133893   -0.0906553   -0.0583811       0.0672755   -0.0187294
 -0.0237936   -0.0340595    0.0393902      -0.0453405    0.0504514
 -0.0648887    0.00188695   0.00248847  …   0.00884857   0.0410842
 -0.0425794   -0.0547981    0.0784          0.0407478   -0.0180079
 -0.00104564  -0.0467049   -0.0858344       0.00935092   0.0966249
  ⋮                                     ⋱               
 -0.060691    -0.00290695   0.0753374       

600×200 Array{Float64,2}:
 -0.0218308   -0.00600274  -0.0877685   …  -0.0275777    -0.0181406
 -0.0187938   -0.0604356   -0.0513703       0.0813879     0.0400525
 -0.00292182  -0.0236161   -0.0672253      -0.0646949    -0.026167
 -0.0320766    0.00996783  -0.0190459       0.0123922    -0.0436413
 -0.0523644   -0.0437903    0.0422649      -0.0201705    -0.00281764
 -0.0560545   -0.0201515    0.0163621   …  -0.000560198  -0.0899267
 -0.0280973   -0.0357981    0.0301558      -0.0659816     0.0245865
 -0.0308715   -0.067648     0.0025547      -0.00426471    0.0275778
 -0.0133893   -0.0906553   -0.0583811      -0.015446      0.0134348
 -0.0237936   -0.0340595    0.0393902      -0.0588948     0.0199524
 -0.0648887    0.00188695   0.00248847  …  -0.0353354    -0.00136956
 -0.0425794   -0.0547981    0.0784          0.0760237    -0.0611772
 -0.00104564  -0.0467049   -0.0858344      -0.0297034     0.0199021
  ⋮                                     ⋱                
 -0.060691    -0.00290695   0.0